# Scripts para crear los archivos txt con los inserts

Importanción de librerías:

In [83]:
import pandas as pd
import numpy as np
from itertools import chain
from database import Database

db = Database('../db/fifa25.db')

Ruta global y lectura de los datasets

In [84]:
BASE_PATH = './../docs/'
df_famales = pd.read_csv(f'{BASE_PATH}female_players.csv', sep=',')
df_males = pd.read_csv(f'{BASE_PATH}male_players.csv', sep=',')
df_all = pd.read_csv(f'{BASE_PATH}all_players.csv', sep=',')
df_all.columns

/tmp/ipykernel_33613/516013953.py:4: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.read_csv(f'{BASE_PATH}all_players.csv', sep=',')


Index(['id', 'Rank', 'Name', 'OVR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY',
       'Acceleration', 'Sprint Speed', 'Positioning', 'Finishing',
       'Shot Power', 'Long Shots', 'Volleys', 'Penalties', 'Vision',
       'Crossing', 'Free Kick Accuracy', 'Short Passing', 'Long Passing',
       'Curve', 'Dribbling', 'Agility', 'Balance', 'Reactions', 'Ball Control',
       'Composure', 'Interceptions', 'Heading Accuracy', 'Def Awareness',
       'Standing Tackle', 'Sliding Tackle', 'Jumping', 'Stamina', 'Strength',
       'Aggression', 'Position', 'Weak foot', 'Skill moves', 'Preferred foot',
       'Height', 'Weight', 'Alternative positions', 'Age', 'Nation', 'League',
       'Team', 'play style', 'url', 'GK Diving', 'GK Handling', 'GK Kicking',
       'GK Positioning', 'GK Reflexes', 'Unnamed: 57'],
      dtype='object')

#### Llenar play_styles

Vamos a leer los <b>play style</b> y los vamos a colocar en una lista. Esta lista no deberá incluir repetidos. Generar un .txt con los inserts con el contenido de la lista creada para usarlos en esta tabla.

```sql
CREATE TABLE play_styles (
  id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  name	VARCHAR(30)
);
```

In [85]:
rows = df_all['play style'].str.split(',')
play_styles = []
for array_str in rows:
    if isinstance(array_str, list):
        for style in array_str:
            # remove '* '
            if style[0] == ' ':
                style = style[1:]
            # remove ' *'
            if style[len(style) - 1] == ' ':
                style = style[:-1]
            if style not in play_styles:
                play_styles.append(style)
text = '-- migrate:up \n\n'
i = 1
for style in play_styles:
    text = text + f"INSERT INTO play_styles (id, name) VALUES ({i}, '{style}');\n"
    i = i + 1
text = text + '\n-- migrate:down \n\nDELETE FROM play_styles;'
with open(f'{BASE_PATH}play_styles_inserts.sql', 'w') as archivo:
    archivo.write(text)

#### Llenar nations

Vamos a leer los <b>natios</b> y los vamos a colocar en una lista. Esta lista no deberá incluir repetidos. Generar un .txt con los inserts con el contenido de la lista creada para usarlos en esta tabla.

```sql
CREATE TABLE nations (
  id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  name	VARCHAR(40)
);
```

In [86]:
nations = sorted(list(df_all['Nation'].unique()))
text = '-- migrate:up \n\n'
i = 1
for name in nations:
    text += "INSERT INTO nations (id, name) VALUES (" + str(i) + ", '" + name + "');\n"
    i += 1
text += '\n-- migrate:down \n\nDELETE FROM nations;'

with open(f'{BASE_PATH}nations_inserts.sql', 'w') as archivo:
    archivo.write(text)

#### Llenar positions

Vamos a leer los <b>positions</b> y los vamos a colocar en una lista. Esta lista no deberá incluir repetidos. Generar un .txt con los inserts con el contenido de la lista creada para usarlos en esta tabla.

```sql
CREATE TABLE positions (
  id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  name	VARCHAR(40)
);
```

In [87]:
# 'Position'  'Alternative positions'

positions = sorted(list(df_all['Position'].unique()))
positions
alternative_positions = df_all['Alternative positions']

rows = df_all['Alternative positions'].str.split(',')
for array_str in rows:
    if isinstance(array_str, list):
        for position in array_str:
            # remove '* '
            if position[0] == ' ':
                position = position[1:]
            # remove ' *'
            if position[len(position) - 1] == ' ':
                position = position[:-1]
            if position not in positions:
                positions.append(position)

text = '-- migrate:up \n\n'
i = 1
for name in positions:
    text += "INSERT INTO positions (id, name) VALUES (" + str(i) + ", '" + name.replace("'", "''") + "');\n"
    i = i + 1
text = text + '\n-- migrate:down \n\nDELETE FROM positions;'
with open(f'{BASE_PATH}positions_inserts.sql', 'w') as archivo:
    archivo.write(text)
    

#### Llenar leagues

Vamos a leer los <b>leagues</b> y los vamos a colocar en una lista. Esta lista no deberá incluir repetidos. Generar un .txt con los inserts con el contenido de la lista creada para usarlos en esta tabla.

```sql
CREATE TABLE leagues (
  id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  name	VARCHAR(30),
  nation_id	INTEGER,
  FOREIGN KEY (nation_id) REFERENCES nations (id)
);
```

In [88]:
df_leagues_nations = pd.read_csv(f'{BASE_PATH}league_nation.csv', sep=',')
leagues = sorted(list(df_all['League'].unique()))
text = '-- migrate:up \n\n'
i = 1
for index, row in df_leagues_nations.iterrows():
    rs = db.fetchone(f"SELECT id FROM nations WHERE name = '{row['Nation']}'")
    if rs is not None:
        nation_id = rs['id']
        name = row['League']
        text += "INSERT INTO leagues (id, name, nation_id) VALUES (" + str(i) + ", '" + name.replace("'", "''") + "', " + str(nation_id) + ");\n"
        i = i + 1
        #print(f"Index: {index}, League: {row['League']}, Nation: {row['Nation']}, NationID: {nation_id}")
    else:
        print(f"Nation '{row['Nation']}' not found in the database.")
text = text + '\n-- migrate:down \n\nDELETE FROM leagues;'
with open(f'{BASE_PATH}leagues_inserts.sql', 'w', encoding='utf-8') as archivo:
    archivo.write(text)

Conexión establecida.


#### Llenar teams

Vamos a leer los <b>teams</b> y los vamos a colocar en una lista. Esta lista no deberá incluir repetidos. Generar un .txt con los inserts con el contenido de la lista creada para usarlos en esta tabla.

```sql
CREATE TABLE teams (
  id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  name	VARCHAR(40),
  league_id	INTEGER NOT NULL,
  FOREIGN KEY (league_id) REFERENCES leagues (id)
);
```

In [106]:

teams_leagues = df_all[['id','Team', 'League']].to_numpy()
teams_list = []
for tl in teams_leagues:
  team = tl[1]
  league = tl[2].replace("'", "''")
  query = f"SELECT id FROM leagues WHERE name = '{league}'"
  rs = db.fetchone(query)
  i = 1
  if rs is not None:
    league_id = rs['id']
    if len(str(league_id)) == 1:
      league_id = '0' + str(league_id)
    else:
      league_id = str(league_id)
    tmp = league_id + '::' + team + '::' + league
    if tmp not in teams_list:
      teams_list.append(tmp)
  else:
      print(f"Leage '{league}' not found in the database.")
teams_list.sort()
# inserts
i = 1
text = '-- migrate:up \n\n'
for t in teams_list:
  data_list = t.split('::')
  team = data_list[1]
  league = data_list[2]
  league_id = int(data_list[0])
  text += "INSERT INTO teams (id, name, league_id) VALUES (" + str(i) + ", '" + team.replace("'", "''") + "', " + str(league_id) + ");\n"
  i = i + 1
text = text + '\n-- migrate:down \n\nDELETE FROM teams;'
with open(f'{BASE_PATH}teams_inserts.sql', 'w', encoding='utf-8') as archivo:
    archivo.write(text)
